## Get the Metadata for the 990 IRS files

In [ ]:
# METHOD 1 - Running outside of Jupyter Notebook should do this.
#try:
#    import import_ipynb
#except:
#    pass
#import IRS990libs

# METHOD 2 - This will reaload without restarting notebook (debugging)
%run IRS990libs.ipynb

In [ ]:
# Set the YEAR of IRS 990 data that we want to download

year = 2017

In [ ]:
# Load the object that parses the IRS 990 index and will provide
# each individual form to us

documents = classy990forms(config='validElements.conf', mongo='mongodb://mongo/irs990', debug=False)
documents.year( year )
documents

In [ ]:
# Loop over all forms and upload to the database.

for x in documents:
    
    sys.stdout.write('Dowload EIN:%s\r' % x.index)
    sys.stdout.flush() 
    
    x.to_mongo()
